In [3]:
import json

In [4]:
with open('./models/1/vocab.json', 'r') as file:
    vocab = json.load(file)

with open('./models/1/reverse_vocab.json', 'r') as file:
    reverse_vocab = json.load(file)

In [6]:
import torch
from src.data_processing import preprocess_and_encode_string

converted_text = preprocess_and_encode_string('dawno dawno temu za górami za lasami', vocab)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load(f'./models/1/model_after_30_epoch.pth')
model.eval()
model.to(device)

MyLSTM(
  (decoder): Decoder(
    (dropout_layer): Dropout(p=0.1, inplace=False)
    (embedding_layer): Embedding(22231, 300)
    (lstm_layer): LSTM(300, 1024, num_layers=2, dropout=0.1)
    (fc): Linear(in_features=1024, out_features=22231, bias=True)
  )
)

In [19]:
from src.data_processing import pad_zeros

input_data = pad_zeros(converted_text[:-1], 50, front=True)

input_tensor = torch.tensor([input_data], dtype=torch.long)
# print(input_tensor.shape)
input_tensor = input_tensor.to(device)
output = model.decoder(input_tensor[-1])
# output = output[1:].reshape(-1, output.shape[2])
# output.argmax(1)
argmax_values, argmax_indices = torch.max(output, dim=-1)
print(argmax_indices, argmax_values)
fairytale.append(argmax_indices.tolist()[0][len_input_data - 1])
# argmax_indices, argmax_values
# decoded_sentence = decode_string(argmax_indices.tolist()[0], reverse_vocab)
# ' '.join(decoded_sentence)

tensor([  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
          2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
          2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
          3,   5,   5,   6, 153,  26, 153,  26], device='cuda:0') tensor([18.0062, 18.0062, 18.0062, 18.0062, 18.0062, 18.0062, 18.0062, 18.0062,
        18.0062, 18.0062, 18.0062, 18.0062, 18.0062, 18.0062, 18.0062, 18.0062,
        18.0062, 18.0062, 18.0062, 18.0062, 18.0062, 18.0062, 18.0062, 18.0062,
        18.0062, 18.0062, 18.0062, 18.0062, 18.0062, 18.0062, 18.0062, 18.0062,
        18.0062, 18.0062, 18.0062, 18.0062, 18.0062, 18.0062, 18.0062, 18.0062,
        18.0062, 18.0062,  5.9264, 11.2564, 11.2564, 10.8817,  9.0546,  9.2523,
         9.0546,  9.2618], device='cuda:0', grad_fn=<MaxBackward0>)


In [31]:
from src.data_processing import decode_string, postprocess_string
from copy import deepcopy

context_len = 50
how_many_sentences = 20
roulette = True

fairytale = deepcopy(converted_text)
# fairytale = [196, 1028,  389,  810, 1029,  170,  992,  131,   26, 1030,  305, 1031,
#      984,    3,    2,  982, 1032,   62, 1033, 1034, 1035,   26,   58,    6,
#      334,  883, 1027,    3,    2,  170, 1036,  193,   98,  908,   34, 1037,
#     1038,  108,  578, 1021,    3,    2,  385,   76,    6,  533,  511, 1039,
#      108, 1040,   44,  533, 1041,    3,    2, 1042, 1043, 1044,   26, 1045,
#      415,  288,  403, 1046,  196,  425,   78, 1047,  108, 1048, 1049,  864,
#     1050,   78,  108, 1051,    3,    2,  982,   61,   62, 1052, 1027,   26,
#     1053,   62,  820,    3,    2, 1012, 1054,   26,   64, 1055, 1056,  236,
#     1057,   31,  218]

while True:
    if how_many_sentences == fairytale.count(3):
        break
    if len(fairytale) > 500:
        break

    input_data = fairytale[-context_len:]
    len_input_data = len(input_data)
    if len(input_data) < context_len:
        input_data = pad_zeros(input_data[:-1], context_len, front=True)

    input_tensor = torch.tensor([input_data], dtype=torch.long)

    input_tensor = input_tensor.to(device)
    output = model.decoder(input_tensor[-1])

    argmax_values, argmax_indices = torch.max(output, dim=-1)

    if roulette:
        topk_values, topk_indices = torch.topk(output, k=10, dim=-1)
        softmax_probs = torch.softmax(topk_values, dim=-1)
        samples = torch.multinomial(softmax_probs, 1, replacement=True)
        selected_indices = topk_indices.gather(1, samples)
        fairytale.append(selected_indices.reshape((1, context_len)).tolist()[0][len_input_data - 1])
    else:
        argmax_values, argmax_indices = torch.max(output, dim=-1)
        fairytale.append(argmax_indices.tolist()[len_input_data - 1])

decoded_sentence = decode_string(fairytale, reverse_vocab)
postprocessed_sentence = postprocess_string(decoded_sentence)
print(postprocessed_sentence)

Dawno dawno temu za górami za lasami.  I dawno pomoc to do po był po.  Się w był i że swojego bardzo Do sobie w i swojego. .  Był Po kilku minutach dotarli wielu przyjaciół i nie jest dla Na to że że w Się że jest to się w lesie i.  Od tamtego dnia w i tak dzięki swojej wioski zaczęli tańczyć w.  I się Po całym królestwie zwierząt a potem wieść w magicznym lesie. . . .  W Się do lasu który uwielbiał spędzać czas na skraju lasu mieszkał w którym zawsze miał swoje marzenia mogą być.  W którym wszyscy byli się że.  Był w końcu gdy jednej chwili pojawił się i tak więc na skraju lasu.  Na to w lesie i i.  I Pewnego pięknego letniego dnia.  I że to było tak dzięki nim.  Był I tak oto opowieść o przygodach.  W magicznym lesie.  i.  I. 
